<a href="https://colab.research.google.com/github/victormurcia/CTS_Test/blob/main/CTS_Parser_Development_Dask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting up the Environment

In [74]:
#Ensure that the required packages are installed in the current environment
!pip install numpy --quiet
!pip install pandas --quiet
!pip install scispacy --quiet
!pip install medspacy --quiet
!pip install scispacy --quiet
!pip install seaborn --quiet
!pip install matplotlib --quiet
!pip install "dask[complete]" --quiet
!pip install ipywidgets --quiet
print('\n')

#Check the current build in Google Colab
!cat /etc/*release
print('\n')

#Check CUDA version
!nvcc --version



DISTRIB_ID=Ubuntu
DISTRIB_RELEASE=20.04
DISTRIB_CODENAME=focal
DISTRIB_DESCRIPTION="Ubuntu 20.04.5 LTS"
NAME="Ubuntu"
VERSION="20.04.5 LTS (Focal Fossa)"
ID=ubuntu
ID_LIKE=debian
PRETTY_NAME="Ubuntu 20.04.5 LTS"
VERSION_ID="20.04"
HOME_URL="https://www.ubuntu.com/"
SUPPORT_URL="https://help.ubuntu.com/"
BUG_REPORT_URL="https://bugs.launchpad.net/ubuntu/"
PRIVACY_POLICY_URL="https://www.ubuntu.com/legal/terms-and-policies/privacy-policy"
VERSION_CODENAME=focal
UBUNTU_CODENAME=focal


nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [75]:
#Import the required libraries/packages
#General utilities
import numpy as np
import pandas as pd
import dask.dataframe as dd
import seaborn as sns
import matplotlib.pyplot as plt
import os, random, time,sys
from ipywidgets import widgets, interact, interactive, fixed, interact_manual

#NLP Stuff
import spacy
import scispacy
import medspacy
from medspacy.ner import TargetRule
from medspacy.visualization import visualize_ent

#Enable data to be extracted from my Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## Load the Data

In [76]:
#Load the Synthetic Veteran Suicide Dataset from my Google Drive
data_dir = r'/content/drive/MyDrive/csv_usa_100k/' #Establish location of data
print('The data is located at:')
print(data_dir, '\n')

def find_csv_files(data_dir):
  """
  This function finds the .csv files located in data_dir and returns an alphabetically sorted file list.

  Parameters:
  data_dir (str) : Directory of files
  
  Returns:
  data_list (list) : list of .csv files
  """
  data_list = [f for f in os.listdir(data_dir) if f.endswith('.csv')] #List the .csv files
  print('The data files are:')
  data_list.sort() #Sort the .csv files alphabetically
  print(data_list,'\n')

  #Get names of .csv files without the extension for naming stuff later
  f_names = [s.replace(".csv","") for s in data_list]

  return data_list,f_names

#Get .csv files
data_list,f_names = find_csv_files(data_dir)

The data is located at:
/content/drive/MyDrive/csv_usa_100k/ 

The data files are:
['allergies.csv', 'careplans.csv', 'conditions.csv', 'devices.csv', 'encounters.csv', 'imaging_studies.csv', 'immunizations.csv', 'medications.csv', 'observations.csv', 'organizations.csv', 'patients.csv', 'payer_transitions.csv', 'payers.csv', 'procedures.csv', 'providers.csv', 'supplies.csv'] 



Some of these files have information that won't be useful for me so I'll remove them from my list now.  The files that I'll be removing are: encounters, organizations, payer_transitions, payers, and providers. 

In [77]:
files2remove = ['encounters.csv', 'organizations.csv', 'payer_transitions.csv', 'payers.csv', 'providers.csv'] 
def remove_files(data_list,files2remove):
  for f in data_list:
    if f in files2remove:
      data_list.remove(f)

  res = filter(lambda i: i not in files2remove, data_list)
  return list(res)

data_list = remove_files(data_list,files2remove)
print(data_list)

['allergies.csv', 'careplans.csv', 'conditions.csv', 'devices.csv', 'imaging_studies.csv', 'immunizations.csv', 'medications.csv', 'observations.csv', 'patients.csv', 'procedures.csv', 'supplies.csv']


In [78]:
#Add data path to beginning of each element of list so that I can easily access them later
def prepend(data_list, str):
    str += '% s'
    new_list = [str % i for i in data_list]
    return new_list

#Add 
csv_list = prepend(data_list,data_dir)
print('Full path of .csv files \n')
csv_list

Full path of .csv files 



['/content/drive/MyDrive/csv_usa_100k/allergies.csv',
 '/content/drive/MyDrive/csv_usa_100k/careplans.csv',
 '/content/drive/MyDrive/csv_usa_100k/conditions.csv',
 '/content/drive/MyDrive/csv_usa_100k/devices.csv',
 '/content/drive/MyDrive/csv_usa_100k/imaging_studies.csv',
 '/content/drive/MyDrive/csv_usa_100k/immunizations.csv',
 '/content/drive/MyDrive/csv_usa_100k/medications.csv',
 '/content/drive/MyDrive/csv_usa_100k/observations.csv',
 '/content/drive/MyDrive/csv_usa_100k/patients.csv',
 '/content/drive/MyDrive/csv_usa_100k/procedures.csv',
 '/content/drive/MyDrive/csv_usa_100k/supplies.csv']

## Create Dataframes With Dask

In [79]:
cols = list(pd.read_csv(csv_list[8], nrows =0)) #Only read the column headers
print(cols)

['Id', 'BIRTHDATE', 'DEATHDATE', 'SSN', 'DRIVERS', 'PASSPORT', 'PREFIX', 'FIRST', 'LAST', 'SUFFIX', 'MAIDEN', 'MARITAL', 'RACE', 'ETHNICITY', 'GENDER', 'BIRTHPLACE', 'ADDRESS', 'CITY', 'STATE', 'COUNTY', 'ZIP', 'LAT', 'LON', 'HEALTHCARE_EXPENSES', 'HEALTHCARE_COVERAGE']


In [80]:
#After checking the contents of all the .csv files, I decided that the following columns across all .csv files will not be 
#included during loading
omit_cols = ['START','STOP','ENCOUNTER','BASE_COST','PAYER_COVERAGE','DATE','TYPE','DEATHDATE', 
             'SSN', 'DRIVERS', 'PASSPORT', 'MAIDEN','ADDRESS','FIRST', 'LAST', 'SUFFIX','START_YEAR', 
             'END_YEAR', 'PAYER','BASE_COST','QUANTITY']

In [81]:
def read_all_csvs(csv_list,omit_cols):
  """
  This function reads all the csv files in a list, loads each of them into a dask dataframe, and 
  then places them into a list called df_list that can be accessed later.

  Parameters:
  csv_list (list)  : List containing csv files
  omit_cols (list) : List containing names of columns to be omitted during dataframe loading

  Returns:
  df_list (list) :list containing dask dataframes
  """
  #Start timer
  start_time = time.time()

  #Initialize array that will contain the loaded dataframes
  df_list = []

  #Number of .csv files to load
  nFiles = len(csv_list)

  #Iterate through the csv files and load the dataframes
  for j in range(nFiles):
    cols    = list(pd.read_csv(csv_list[j], nrows = 0)) #Only read header row for column names
    temp_df = dd.read_csv(csv_list[j] , usecols =[i for i in cols if i not in omit_cols],assume_missing=True,dtype={'VALUE': 'object'}) #Make dask df
    df_list.append(temp_df)
  
  #End timer
  print("Loading of " + str(nFiles) + " .csv files took: %.2f seconds" % (time.time() - start_time))

  return df_list

df_list = read_all_csvs(csv_list,omit_cols)

Loading of 11 .csv files took: 0.41 seconds


## EDA

In [82]:
#Visualize descriptions columns of dataframe 
def make_countplots(df,name,n):

  if n < 25:
    val = n
  else:
    val = 25
  b = sns.countplot(data=df,y='DESCRIPTION',order = df['DESCRIPTION'].value_counts().index[:25])
  b.axes.set_title("Top "+ str(val) + ' ' + name.capitalize() + " Present in Dataset",fontsize=25)
  b.set_xlabel("Counts",fontsize=15)
  b.set_ylabel(name.capitalize(),fontsize=15)
  b.tick_params(labelsize=8)
  plt.show()

#View the number of unique patients in each dataframe
@interact
def show_nunique_vals_in_df(x=widgets.IntSlider(min=0,max=len(df_list)-1,step=1,value=0)):
  nPatients = df_list[x]['PATIENT'].nunique().compute()
  nDescriptions = df_list[x]['DESCRIPTION'].nunique().compute()
  print('The ' + f_names[x] + '.csv file has' ,nPatients, 'unique patients and',nDescriptions,'unique',f_names[x])
  make_countplots(df_list[x].compute(),f_names[x],nDescriptions)

interactive(children=(IntSlider(value=0, description='x', max=10), Output()), _dom_classes=('widget-interact',…

## Restructuring and Merging Dataframes

In [118]:
def restructure_dfs(ref_df,source_df,nPatients):
    """
    This function combines the observations for a patient in the Synthetic Veteran Suicide Dataset into a 
    single row in a new dataframe. Doing this should help with the NLP processing later.
    
    Parameters
    ----------
    nPatients : int,  How many patients to process?
    ref_df    : df,   Dataframe containing restructured patient data (should be the first dataframe that was 
                      restructured)
    source_df : df,   Dataframe that contains information to be restructured
    ----------
    """
    # Create empty patient dataframe
    columns = source_df.columns
    temp_df = pd.DataFrame(index=np.arange(nPatients),columns=columns)    
    
    for i in range(ref_df.shape[0]):
        patient = ref_df['PATIENT'][i] #Patient ID    
        #dummy_df   = source_df[source_df['PATIENT'] == patient]
        
        #Programatically populate the dataframe by first making lists out of the columns and then inserting
        #these lists into cells in the resulting dataframe
        for k,col in enumerate(columns):
            if col == 'PATIENT':
                temp_df.at[i, 'PATIENT'] = patient    
            else:
                temp_df.at[i, col] = source_df[col].tolist()
                
    return temp_df

def make_patient_df(nPatients,df,df_list, s_list, init_rand=False):
    """
    This function combines the observations for a patient in the Synthetic Veteran Suicide Dataset into a 
    single row in a new dataframe. Doing this should help with the NLP processing later.
    
    Parameters
    ----------
    nPatients : int,  How many patients to process?
    df        : df,   Dataframe containing patient data
    df_list   : list, list of all dataframes in dataset
    init_rand : bool, Use random values to choose patients? Defaults to False so only first 10 patients are selected
    s_list    : list, List of strings to append to column names in the reformatted dataframes 
    ----------
    """

    #Start timer
    start_time = time.time()

    #Determine number of unique patients in dataframe
    unique_patients = df['PATIENT'].nunique()
    
    #Initialize iterators and patient index
    i = 0; j = random.randrange(unique_patients)
    
    #Initialize index array
    index_list = []
    
    #Make list of restructured dataframes
    rs_df_list = []
    
    for i in range(nPatients):
      # Create empty patient dataframe
      columns = df.columns
      new_df  = pd.DataFrame(index=np.arange(nPatients),columns=columns)
      
      #Select a random patient from the existing dataframe (random sample)
      if init_rand == True:
        j = random.randrange(unique_patients)                
      else:
        j = i 
      
      #Check that patient id has not already been added
      if j not in index_list:
        index_list.append(j)
      else:
        while j in index_list:
            j = random.randrange(unique_patients) 

        index_list.append(j)
      
      #Subset the dataframe so as to only show the results associated with a given patient
      patient    = df['PATIENT'][j] #Patient ID
      dummy_df   = df[df['PATIENT'] == patient]
      
      #Programatically populate the dataframe by first making lists out of the columns and then inserting
      #these lists into cells in the resulting dataframe
      for k,col in enumerate(columns):
        if col == 'PATIENT':
            new_df.at[i, 'PATIENT'] = patient    
        else:
            new_df.at[i, col] = dummy_df[col].tolist()
      
      #Append the restructured dataframe to the list of dfs
      rs_df_list.append(new_df)

      #1. Select the source dataframe to be converted from df_list
      #2. Subset this dataframe so that only the rows with the current patient ids are selected
      for m,cdf in enumerate(df_list[1:3]):
        columns = cdf.columns

        # Create empty patient dataframe
        new_df  = pd.DataFrame(index=np.arange(nPatients),columns=columns)

        if 'PATIENT' not in columns:
          dummy_df = cdf[cdf['Id'] == patient]
        else:
          dummy_df = cdf[cdf['PATIENT'] == patient]

        dummy_df = dummy_df.compute()

        for k,col in enumerate(columns):
          if 'PATIENT' not in columns:
            if col == 'Id':
              new_df.at[i, 'PATIENT'] = patient    
            else:
              new_df.at[i, col] = dummy_df[col].tolist()
          else:
            if col == 'PATIENT':
              new_df.at[i, 'PATIENT'] = patient    
            else:
              new_df.at[i, col] = dummy_df[col].tolist()

        rs_df_list.append(new_df)

    #Rename columns so as to have a suffix included for easy traceback to source file
    for m,df in enumerate(rs_df_list):
      columns = df.columns
      if 'PATIENT' not in columns:
        df.columns = [x + s_list[m] if x != 'Id' else x for x in columns]
      else:
        df.columns = [x + s_list[m] if x != 'PATIENT' else x for x in columns]    

    print('These are the indices for the patient IDs used to generate the dataframe below. \n',index_list)

    #End timer
    print("Reformatting dataframes took: %.2f seconds" % (time.time() - start_time))

    return rs_df_list  

s_list = ['_als','_cps','_cds','_dvs','_iss','_ims','_mds','_obs','_pts','_prs','_sps']
rs_df_list  = make_patient_df(1, df_list[0].compute(),df_list, s_list, init_rand=True)

These are the indices for the patient IDs used to generate the dataframe below. 
 [8594]
Reformatting dataframes took: 6.34 seconds


In [122]:
rs_df_list[0]

,PATIENT,CODE_als,DESCRIPTION_als
0,2791aa30-ec35-47fe-af04-ef9ce5feedb2,"[419474003.0, 232347008.0, 418689008.0, 419263...","[Allergy to mould, Dander (animal) allergy, Al..."


In [123]:
#Merge the dataframes together using the 'PATIENT' column
dfs = [df.set_index('PATIENT') for df in rs_df_list]
df = pd.concat(dfs, axis=1)
df

,CODE_als,DESCRIPTION_als,Id_cps,CODE_cps,DESCRIPTION_cps,REASONCODE_cps,REASONDESCRIPTION_cps,CODE_cds,DESCRIPTION_cds
PATIENT,,,,,,,,,
2791aa30-ec35-47fe-af04-ef9ce5feedb2,"[419474003.0, 232347008.0, 418689008.0, 419263...","[Allergy to mould, Dander (animal) allergy, Al...","[bd5071b3-89de-4999-a139-fc63defc0f00, 0208b9d...","[384758001.0, 91251008.0, 385691007.0, 1708360...","[Self-care interventions (procedure), Physical...","[nan, 70704007.0, 263102004.0, nan, 44465007.0...","[nan, Sprain of wrist, Fracture subluxation of...","[444814009.0, 446096008.0, 128613002.0, 703151...","[Viral sinusitis (disorder), Perennial allergi..."


In [85]:
list(df.columns)

['CODE_als',
 'DESCRIPTION_als',
 'Id_cps',
 'CODE_cps',
 'DESCRIPTION_cps',
 'REASONCODE_cps',
 'REASONDESCRIPTION_cps',
 'CODE_cds',
 'DESCRIPTION_cds']

In [86]:
#I don't need the Id columns so I'll drop them
df2 = df[df.columns.drop(list(df.filter(regex='Id')))]

In [87]:
new_names = ['CODE_als',
 'DESCRIPTION_als',
 'Id_cps',
 'CODE_cps',
 'DESCRIPTION_cps',
 'REASONCODE_cps',
 'REASONDESCRIPTION_cps',
 'CODE_cds',
 'DESCRIPTION_cds',
 'CODE_dvs',
 'DESCRIPTION_dvs',
 'UDI_dvs',
 'Id_iss',
 'BODYSITE_CODE_iss',
 'BODYSITE_DESCRIPTION_iss',
 'MODALITY_CODE_iss',
 'MODALITY_DESCRIPTION_iss',
 'SOP_CODE_iss',
 'SOP_DESCRIPTION_iss',
 'CODE_ims',
 'DESCRIPTION_ims',
 'CODE_mds',
 'DESCRIPTION_mds',
 'DISPENSES_mds',
 'TOTALCOST_mds',
 'REASONCODE_mds',
 'REASONDESCRIPTION_mds',
 'CODE_obs',
 'DESCRIPTION_obs',
 'VALUE_obs',
 'UNITS_obs',
 'Id_pts',
 'BIRTHDATE_pts',
 'PREFIX_pts',
 'MARITAL_pts',
 'RACE_pts',
 'ETHNICITY_pts',
 'GENDER_pts',
 'BIRTHPLACE_pts',
 'CITY_pts',
 'STATE_pts',
 'COUNTY_pts',
 'ZIP_pts',
 'LAT_pts',
 'LON_pts',
 'HEALTHCARE_EXPENSES_pts',
 'HEALTHCARE_COVERAGE_pts',
 'CODE_prs',
 'DESCRIPTION_prs',
 'REASONCODE_prs',
 'REASONDESCRIPTION_prs',
 'CODE_sps',
 'DESCRIPTION_sps']

## Accessing Clinical Trials

In [88]:
#Query for trials
cond = 'allergy'#input('Enter the disease condition to find clinical trials: ')
a = 'https://clinicaltrials.gov/api/query/study_fields?expr='
b = '&fields=NCTId%2CBriefTitle%2CCondition%2COverallStatus%2CLeadSponsorName%2CEligibilityCriteria'
c = '&min_rnk=1&max_rnk=1000&fmt=csv'
q=(a + cond + b + c)
print(q)
qtrials = pd.read_csv(q,skiprows=10)

@interact
def show_recruiting_studies(column=['OverallStatus','Condition'], 
                            x = ['Recruiting','Completed','Unknown status'],
                            y = ['Food','Antibiotic']
                            ):
    if column == 'OverallStatus':
      return qtrials.loc[qtrials[column] == x]
    elif column == 'Condition':
      return qtrials[qtrials['Condition'].str.contains(y)]

https://clinicaltrials.gov/api/query/study_fields?expr=allergy&fields=NCTId%2CBriefTitle%2CCondition%2COverallStatus%2CLeadSponsorName%2CEligibilityCriteria&min_rnk=1&max_rnk=1000&fmt=csv


interactive(children=(Dropdown(description='column', options=('OverallStatus', 'Condition'), value='OverallSta…

In [89]:
qtrials

,Rank,NCTId,BriefTitle,Condition,OverallStatus,LeadSponsorName,EligibilityCriteria
0,1,NCT04827602,Drug Allergy Labels After Drug Allergy Investi...,Drug Hypersensitivity,Completed,"University Hospital, Gentofte, Copenhagen",Inclusion Criteria:||Penicillin allergy tested...
1,2,NCT03826953,Allergy UK Research and Development Nurse Project,Allergy,Unknown status,University of Edinburgh,"Inclusion Criteria:||All children, young peopl..."
2,3,NCT05561777,Penicillin Allergy Risk-Stratification and Del...,Antibiotic Allergy,Recruiting,Vanderbilt University Medical Center,Inclusion Criteria:||Pediatric Hospital Medici...
3,4,NCT01914978,Evaluating the Effectiveness of a Handbook for...,"Hypersensitivity, Food",Completed,Boston Children's Hospital,Inclusion Criteria:||Parents of children ages ...
4,5,NCT03581604,De-labeling of Patients With False Diagnosis o...,Allergy Drug,Recruiting,Oslo University Hospital,Inclusion Criteria:||Adult patients who are re...
...,...,...,...,...,...,...,...
995,996,NCT02378129,Evaluation of Dentin Hypersensitivity Using Gl...,Dentin Hypersensitivity,Completed,Federal University of Pelotas,Inclusion Criteria:||Male and female subjects ...
996,997,NCT02060864,Effect of AN-PEP Enzyme on Gluten Digestion in...,Non-coeliac Gluten Sensitivity,Completed,DSM Food Specialties,Inclusion Criteria:||Male/female|Age ≥18 but <...
997,998,NCT05704218,Hypersensitivity Pneumonitis of Domestic Origin,Sensitisation|Mold or Dust Allergy|Respiratory...,Not yet recruiting,Centre Hospitalier Universitaire de Besancon,Inclusion Criteria:||exposure to mold at home|...
998,999,NCT03703791,"Real World, Open Label, QOL Assessment of Pean...",Peanut Allergy,Terminated,"Aimmune Therapeutics, Inc.",Key Inclusion Criteria:||Age 4 through 17 year...
